In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_feather('D:/big-datasets/internship-telkom/eda-tiket-no-missing.ftr')

In [3]:
df

,reportdate,is_gamas,emosi,datel,cwitel,reg,speed_id,lcat,bisa_browsing,gangguan_v2
0,2021-06-19 12:30:31,0,Ramah,Nunukan,47,6,3M,Residensial,0,2P / 3P Mati Total
1,2021-05-24 14:32:10,1,Ramah,Balikpapan,45,6,10M,Prime Cluster,0,Tidak Bisa Koneksi
2,2021-04-04 11:27:17,0,Ramah,Jambi,6,1,10M,Residensial,0,2P / 3P Mati Total
3,2021-06-26 21:09:49,0,Ramah,Kujang,19,2,20M,Prime Cluster,1,Lambat
4,2021-06-15 11:02:32,0,Marah,Blitar,35,5,20M,Residensial,1,Lambat
...,...,...,...,...,...,...,...,...,...,...
2632496,2021-01-04 12:21:16,0,Marah,Sumedang,23,3,20M,Prime Cluster,0,2P / 3P Mati Total
2632497,2021-02-18 09:56:08,0,Marah,Nganjuk,35,5,10M,Residensial,1,Lambat
2632498,2021-05-29 08:31:46,0,Ramah,Garut,24,3,20M,Residensial,0,2P / 3P Mati Total
2632499,2021-05-04 08:22:38,0,Marah,Surabaya selatan,38,5,30M,Residensial,1,Lambat


In [7]:
def emosi_summary_gangguan(thn, bln, cw):
    ex = df[(df['reportdate'].dt.year == thn) & (df['reportdate'].dt.month == bln) & (df['cwitel'] == cw)]
    for gangguan in ex['gangguan_v2'].value_counts().head().index:
        ramah_perc = round(ex['emosi'][ex['gangguan_v2'] == gangguan].value_counts(normalize=True) * 100, 2).to_dict()['Ramah']
        marah_perc = round(ex['emosi'][ex['gangguan_v2'] == gangguan].value_counts(normalize=True) * 100, 2).to_dict()['Marah']
        text = f'''
        Gangguan: {gangguan}
        Persentase Respon Ramah: {ramah_perc}%
        Persentase Respon Marah: {marah_perc}%
        '''
        yield text
        
generator = emosi_summary_gangguan(thn=2021, bln=1, cw=1)
for i in generator:
    print(i)


        Gangguan: Tidak Bisa Koneksi
        Persentase Respon Ramah: 76.92%
        Persentase Respon Marah: 23.08%
        

        Gangguan: 2P / 3P Mati Total
        Persentase Respon Ramah: 70.31%
        Persentase Respon Marah: 29.69%
        

        Gangguan: Lambat
        Persentase Respon Ramah: 53.0%
        Persentase Respon Marah: 47.0%
        

        Gangguan: Intermitten / Putus-Putus
        Persentase Respon Ramah: 60.95%
        Persentase Respon Marah: 39.05%
        

        Gangguan: Tidak Bisa ke Website Tertentu
        Persentase Respon Ramah: 66.67%
        Persentase Respon Marah: 33.33%
        


In [8]:
def emosi_summary_jam(thn, bln, cw):
    ex = df[(df['reportdate'].dt.year == thn) & (df['reportdate'].dt.month == bln) & (df['cwitel'] == cw)]
    # 0-6
    ramah_06 = round(ex['emosi'][(ex['reportdate'].dt.hour >= 0) & (ex['reportdate'].dt.hour < 6)].value_counts(normalize=True) * 100, 2).to_dict()['Ramah']
    marah_06 = round(ex['emosi'][(ex['reportdate'].dt.hour >= 0) & (ex['reportdate'].dt.hour < 6)].value_counts(normalize=True) * 100, 2).to_dict()['Marah']
    # 6-12
    ramah_612 = round(ex['emosi'][(ex['reportdate'].dt.hour >= 6) & (ex['reportdate'].dt.hour < 12)].value_counts(normalize=True) * 100, 2).to_dict()['Ramah']
    marah_612 = round(ex['emosi'][(ex['reportdate'].dt.hour >= 6) & (ex['reportdate'].dt.hour < 12)].value_counts(normalize=True) * 100, 2).to_dict()['Marah']
    # 12-18
    ramah_1218 = round(ex['emosi'][(ex['reportdate'].dt.hour >= 12) & (ex['reportdate'].dt.hour < 18)].value_counts(normalize=True) * 100, 2).to_dict()['Ramah']
    marah_1218 = round(ex['emosi'][(ex['reportdate'].dt.hour >= 12) & (ex['reportdate'].dt.hour < 18)].value_counts(normalize=True) * 100, 2).to_dict()['Marah']
    # 18-23
    ramah_1823 = round(ex['emosi'][(ex['reportdate'].dt.hour >= 18) & (ex['reportdate'].dt.hour <= 23)].value_counts(normalize=True) * 100, 2).to_dict()['Ramah']
    marah_1823 = round(ex['emosi'][(ex['reportdate'].dt.hour >= 18) & (ex['reportdate'].dt.hour <= 23)].value_counts(normalize=True) * 100, 2).to_dict()['Marah']

    # print text
    text = f'''
    Pukul 00:00 - 06:00
    Persentase Respon Ramah: {ramah_06}
    Persentase Respon Marah: {marah_06}

    Pukul 06:00 - 12:00
    Persentase Respon Ramah: {ramah_612}
    Persentase Respon Marah: {marah_612}

    Pukul 12:00 - 18:00
    Persentase Respon Ramah: {ramah_1218}
    Persentase Respon Marah: {marah_1218}

    Pukul 18:00 - 24:00
    Persentase Respon Ramah: {ramah_1823}
    Persentase Respon Marah: {marah_1823}
    '''
    yield text

    
generator = emosi_summary_jam(thn=2021, bln=1, cw=1)
for i in generator:
    print(i)


    Pukul 00:00 - 06:00
    Persentase Respon Ramah: 78.02
    Persentase Respon Marah: 21.98

    Pukul 06:00 - 12:00
    Persentase Respon Ramah: 74.18
    Persentase Respon Marah: 25.82

    Pukul 12:00 - 18:00
    Persentase Respon Ramah: 69.47
    Persentase Respon Marah: 30.53

    Pukul 18:00 - 24:00
    Persentase Respon Ramah: 62.09
    Persentase Respon Marah: 37.91
    
